### Data Loading

In [1]:
import pandas as pd
import numpy as np
from google.cloud import bigquery as bq

service_account_path = "/home/yusuf/DataScience/dream_games/ybektas20.json" 
client = bq.Client.from_service_account_json(service_account_path)

# Define queries for AB test tables.
ab_test_tables_queries = {
    "q2_table_ab_test_enter": """
        SELECT
          COUNT(*) AS total_rows,
          COUNT(test_entry_timestamp) AS non_null_test_entry_timestamp,
          COUNT(install_timestamp) AS non_null_install_timestamp,
          COUNT(user_id) AS non_null_user_id,
          COUNT(platform) AS non_null_platform,
          COUNT(group_id) AS non_null_group_id
        FROM `casedreamgames.case_db.q2_table_ab_test_enter`;
    """,
    "q2_table_ab_test_revenue": """
        SELECT
          COUNT(*) AS total_rows,
          COUNT(event_timestamp) AS non_null_event_timestamp,
          COUNT(user_id) AS non_null_user_id,
          COUNT(platform) AS non_null_platform,
          COUNT(package_type) AS non_null_package_type,
          COUNT(level) AS non_null_level,
          COUNT(dollar_amount) AS non_null_dollar_amount
        FROM `casedreamgames.case_db.q2_table_ab_test_revenue`;
    """,
    "q2_table_ab_test_session": """
        SELECT
          COUNT(*) AS total_rows,
          COUNT(event_timestamp) AS non_null_event_timestamp,
          COUNT(user_id) AS non_null_user_id,
          COUNT(platform) AS non_null_platform,
          COUNT(time_spent) AS non_null_time_spent,
          COUNT(level) AS non_null_level
        FROM `casedreamgames.case_db.q2_table_ab_test_session`;
    """
}

# Iterate through each query, execute it, and print the shape and null counts per column.
for table_name, query in ab_test_tables_queries.items():
    print(f"Results for {table_name}:")
    
    # Run the query; assumes 'client' is your configured BigQuery client.
    df = client.query(query).result().to_dataframe()
    
    # Extract total row count.
    total_rows = df.loc[0, "total_rows"]
    print(f"Shape: ({total_rows} rows)")
    
    # For each column (ignoring the total_rows column), compute and print null counts.
    for col in df.columns:
        if col != "total_rows":
            non_null_count = df.loc[0, col]
            null_count = total_rows - non_null_count
            # Remove the "non_null_" prefix to display the original column name.
            orig_col = col.replace("non_null_", "")
            print(f"Column '{orig_col}': non-null = {non_null_count}, null = {null_count}")
    
    print("\n" + "-"*50 + "\n")


Results for q2_table_ab_test_enter:


/home/yusuf/myenv/lib/python3.12/site-packages/google/cloud/bigquery/table.py:1820: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


Shape: (73450 rows)
Column 'test_entry_timestamp': non-null = 73450, null = 0
Column 'install_timestamp': non-null = 72249, null = 1201
Column 'user_id': non-null = 73450, null = 0
Column 'platform': non-null = 73450, null = 0
Column 'group_id': non-null = 73450, null = 0

--------------------------------------------------

Results for q2_table_ab_test_revenue:
Shape: (74929 rows)
Column 'event_timestamp': non-null = 74929, null = 0
Column 'user_id': non-null = 74929, null = 0
Column 'platform': non-null = 74929, null = 0
Column 'package_type': non-null = 74929, null = 0
Column 'level': non-null = 74929, null = 0
Column 'dollar_amount': non-null = 74929, null = 0

--------------------------------------------------

Results for q2_table_ab_test_session:
Shape: (224144299 rows)
Column 'event_timestamp': non-null = 224144299, null = 0
Column 'user_id': non-null = 224144299, null = 0
Column 'platform': non-null = 224144246, null = 53
Column 'time_spent': non-null = 224144296, null = 3
Colu

### Eda

Is the distribution of users by groups and platforms balanced?

In [2]:
query_users = """
    SELECT 
        group_id,
        platform, 
        COUNT(DISTINCT user_id) AS user_count
    FROM `casedreamgames.case_db.q2_table_ab_test_enter`
    GROUP BY group_id, platform
    ORDER BY group_id, platform;
"""
df_users = client.query(query_users).result().to_dataframe()
print("User Count by Group:")
df_users


User Count by Group:


,group_id,platform,user_count
0,A,android,9587
1,A,ios,27601
2,B,android,9539
3,B,ios,26723


users seem to be evenly distributed across the groups and platforms.

How does total and average time spent to the game by each user differ between groups?

In [3]:
# Q3: Engagement metrics by test group (aggregated session data)
query_session_time = """
    SELECT 
        e.group_id,
        e.platform,
        COUNT(DISTINCT s.user_id) AS users_with_sessions,
        AVG(s.total_time_spent) AS avg_time_spent,
        AVG(s.session_count) AS avg_session_count
    FROM (
        SELECT 
            user_id, 
            SUM(time_spent) AS total_time_spent,
            COUNT(*) AS session_count
        FROM `casedreamgames.case_db.q2_table_ab_test_session`
        GROUP BY user_id
    ) s
    JOIN `casedreamgames.case_db.q2_table_ab_test_enter` e
      ON s.user_id = e.user_id
    GROUP BY e.group_id, e.platform
    ORDER BY e.group_id, e.platform;
"""
df_time_spend = client.query(query_session_time).result().to_dataframe()
df_time_spend['avg_time_per_session'] = df_time_spend['avg_time_spent'] / df_time_spend['avg_session_count']
print("Session Engagement Metrics by Group:")
df_time_spend


/home/yusuf/myenv/lib/python3.12/site-packages/google/cloud/bigquery/table.py:1820: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


Session Engagement Metrics by Group:


,group_id,platform,users_with_sessions,avg_time_spent,avg_session_count,avg_time_per_session
0,A,android,9560,96423.673640,3302.667887,29.195692
1,A,ios,27517,101195.179053,3510.551514,28.826006
2,B,android,9503,69394.566768,2387.807114,29.062049
3,B,ios,26619,78842.170630,2752.906458,28.639611


it seems that for both platforms, the group A users spent more time and played more levels compared to the control group. Also their average time per session is also higher

Now we will inspect the level proogression of users

In [4]:
query_level_progression_by_group = """
    SELECT 
        e.group_id,
        e.platform,
        COUNT(DISTINCT s.user_id) AS users_with_sessions,
        AVG(s.max_level) AS avg_max_level_reached
    FROM (
        SELECT 
            user_id, 
            MAX(level) AS max_level
        FROM `casedreamgames.case_db.q2_table_ab_test_session`
        GROUP BY user_id
    ) s
    JOIN `casedreamgames.case_db.q2_table_ab_test_enter` e 
      ON s.user_id = e.user_id
    GROUP BY e.group_id, e.platform
    ORDER BY e.group_id, e.platform;
"""
df_level_progression = client.query(query_level_progression_by_group).result().to_dataframe()
print("Average Maximum Level Reached by AB Test Group:")
df_level_progression

/home/yusuf/myenv/lib/python3.12/site-packages/google/cloud/bigquery/table.py:1820: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


Average Maximum Level Reached by AB Test Group:


,group_id,platform,users_with_sessions,avg_max_level_reached
0,A,android,9560,254.458264
1,A,ios,27517,293.575499
2,B,android,9503,133.699148
3,B,ios,26619,161.844134


Group B users played significantly less levels than A. In both groups, ios players have higher level progresion.

Now, we will investigate retention rate for 1 day

In [5]:
retention_query = """
WITH test_entries AS (
  SELECT
    user_id,
    group_id,
    DATE(test_entry_timestamp) AS entry_date
  FROM `casedreamgames.case_db.q2_table_ab_test_enter`
),
one_day_sessions AS (
  SELECT DISTINCT
    user_id,
    DATE(event_timestamp) AS session_date
  FROM `casedreamgames.case_db.q2_table_ab_test_session`
)
SELECT
  te.entry_date,
  te.group_id,
  COUNT(DISTINCT te.user_id) AS total_users,
  COUNT(DISTINCT CASE 
                   WHEN ods.session_date = DATE_ADD(te.entry_date, INTERVAL 1 DAY) 
                   THEN te.user_id 
                 END) AS retained_users,
  COUNT(DISTINCT CASE 
                   WHEN ods.session_date = DATE_ADD(te.entry_date, INTERVAL 1 DAY) 
                   THEN te.user_id 
                 END) * 1.0 / COUNT(DISTINCT te.user_id) AS retention_rate
FROM test_entries te
LEFT JOIN one_day_sessions ods
  ON te.user_id = ods.user_id 
GROUP BY te.entry_date, te.group_id
ORDER BY te.entry_date, te.group_id;
"""

df_retention = client.query(retention_query).result().to_dataframe()

import plotly.express as px


fig = px.line(df_retention, x='entry_date', y='retention_rate', color='group_id', title='1-Day Retention Rate by Group')
fig.show()
print("1-Day Retention Rate by Group: Descriptive Statistics")

df_retention.groupby('group_id')['retention_rate'].describe()


/home/yusuf/myenv/lib/python3.12/site-packages/google/cloud/bigquery/table.py:1820: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


1-Day Retention Rate by Group: Descriptive Statistics


,count,mean,std,min,25%,50%,75%,max
group_id,,,,,,,,
A,29.0,0.582474,0.027382,0.534738,0.570767,0.580838,0.589676,0.681892
B,29.0,0.570840,0.028710,0.500711,0.557978,0.577736,0.587819,0.642743


In [6]:

# Query for 7-day retention
retention_query_7day = """
WITH test_entries AS (
  SELECT
    user_id,
    group_id,
    DATE(test_entry_timestamp) AS entry_date
  FROM `casedreamgames.case_db.q2_table_ab_test_enter`
),
seven_day_sessions AS (
  SELECT DISTINCT
    user_id,
    DATE(event_timestamp) AS session_date
  FROM `casedreamgames.case_db.q2_table_ab_test_session`
)
SELECT
  te.entry_date,
  te.group_id,
  COUNT(DISTINCT te.user_id) AS total_users,
  COUNT(DISTINCT CASE 
                   WHEN seven_day_sessions.session_date = DATE_ADD(te.entry_date, INTERVAL 7 DAY) 
                   THEN te.user_id 
                 END) AS retained_users,
  COUNT(DISTINCT CASE 
                   WHEN seven_day_sessions.session_date = DATE_ADD(te.entry_date, INTERVAL 7 DAY) 
                   THEN te.user_id 
                 END) * 1.0 / COUNT(DISTINCT te.user_id) AS retention_rate
FROM test_entries te
LEFT JOIN seven_day_sessions
  ON te.user_id = seven_day_sessions.user_id 
GROUP BY te.entry_date, te.group_id
ORDER BY te.entry_date, te.group_id;
"""

# Run the query and load into a DataFrame
df_retention_7day = client.query(retention_query_7day).result().to_dataframe()

# Plot the 7-day retention rate by test group over time using Plotly
import plotly.express as px

fig_7day = px.line(
    df_retention_7day,
    x='entry_date',
    y='retention_rate',
    color='group_id',
    title='7-Day Retention Rate by Group',
    labels={
        "entry_date": "Test Entry Date",
        "retention_rate": "7-Day Retention Rate",
        "group_id": "Test Group"
    }
)
fig_7day.show()

print("7-Day Retention Rate by Group: Descriptive Statistics")
print(df_retention_7day.groupby('group_id')['retention_rate'].describe())


/home/yusuf/myenv/lib/python3.12/site-packages/google/cloud/bigquery/table.py:1820: UserWarning:

BigQuery Storage module not found, fetch data with the REST endpoint instead.



7-Day Retention Rate by Group: Descriptive Statistics
          count      mean       std       min       25%       50%       75%  \
group_id                                                                      
A          29.0  0.427711  0.019208  0.401487  0.412318  0.425517  0.436314   
B          29.0  0.412179  0.016478  0.373847  0.404563  0.410697  0.420810   

               max  
group_id            
A         0.487765  
B         0.451960  


the retention rates also seems better for group A compared to group B.
    
All in all we can say that group A is more successful than group B in terms oof engagement.

### Monetization by groups
Which group generated a higher revenue per user in total?

In [ ]:

"""
package_by_user = df_revenue.groupby(["group_id", "package_type"]).agg(
    total_revenue=("total_revenue", "sum"),
    total_users=("user_id", "nunique"),
    total_purchases=("total_revenue", "count")
).reset_index()

package_by_user
"""

In [30]:
query_revenue = """
WITH test_entries AS (
  SELECT 
    user_id,
    group_id,
    platform
  FROM `casedreamgames.case_db.q2_table_ab_test_enter`
),
user_package AS (
  SELECT
    user_id,
    package_type,
    SUM(dollar_amount) AS total_revenue
  FROM `casedreamgames.case_db.q2_table_ab_test_revenue`
  GROUP BY user_id, package_type
)
SELECT 
  te.user_id,
  te.group_id,
  te.platform,
  COALESCE(up.package_type, 'no_purchase') AS package_type,
  COALESCE(up.total_revenue, 0) AS total_revenue
FROM test_entries te
LEFT JOIN user_package up
  ON te.user_id = up.user_id;
"""

df_revenue = client.query(query_revenue).result().to_dataframe()
df_revenue


/home/yusuf/myenv/lib/python3.12/site-packages/google/cloud/bigquery/table.py:1820: UserWarning:

BigQuery Storage module not found, fetch data with the REST endpoint instead.



,user_id,group_id,platform,package_type,total_revenue
0,txlqw6185177497a2,A,ios,no_purchase,0.0
1,txlqw6185791752a2,A,ios,no_purchase,0.0
2,txlqw6187363919a2,A,ios,no_purchase,0.0
3,txlqw6186918784a2,A,ios,no_purchase,0.0
4,txlqw6186405215a2,A,ios,no_purchase,0.0
...,...,...,...,...,...
79330,txlqw6185500868a2,B,ios,no_purchase,0.0
79331,txlqw6187103070a2,B,ios,no_purchase,0.0
79332,txlqw6185674719a2,B,ios,no_purchase,0.0
79333,txlqw6187314123a2,B,android,no_purchase,0.0


We will make statistical tests to see if the differences between the groups are statistically significant for conversion and arpu.

In [31]:
from statsmodels.stats.proportion import proportions_ztest
from scipy.stats import ttest_ind

df_user = df_revenue.groupby(['user_id', 'group_id'], as_index=False).agg(
    total_revenue = ('total_revenue', 'sum')
)

group_summary = df_user.groupby('group_id').agg(
    total_users = ('user_id', 'count'),
    total_revenue = ('total_revenue', 'sum'),
    converted_users = ('total_revenue', lambda x: (x > 0).sum())
).reset_index()

group_summary['ARPU'] = group_summary['total_revenue'] / group_summary['total_users']
group_summary['conversion_rate'] = group_summary['converted_users'] / group_summary['total_users']

print("\nGroup-level summary:")
print(group_summary)

# --------------------------
# Conversion Rate Statistical Test: Two-Proportion Z-Test
# --------------------------
conv_counts = group_summary.set_index('group_id').loc[['A', 'B'], 'converted_users'].values
conv_nobs = group_summary.set_index('group_id').loc[['A', 'B'], 'total_users'].values

z_stat, p_val = proportions_ztest(conv_counts, conv_nobs)
print("\nTwo-Proportion Z-test for Conversion Rates:")
print(f"Z-statistic: {z_stat:.3f}")
print(f"P-value: {p_val:.5f}")

# --------------------------
# ARPU Statistical Test: Welch's T-Test
# --------------------------
arpu_A = df_user[df_user['group_id'] == 'A']['total_revenue']
arpu_B = df_user[df_user['group_id'] == 'B']['total_revenue']

t_stat, t_p_val = ttest_ind(arpu_A, arpu_B, equal_var=False)
print("\nWelch's T-test for ARPU (Group A vs. Group B):")
print(f"t-statistic: {t_stat:.3f}")
print(f"P-value: {t_p_val:.5f}")

# Optionally, print standard deviations for ARPU per group:
arpu_stats = df_user.groupby('group_id')['total_revenue'].agg(['mean', 'std', 'count']).reset_index()
print("\nARPU Descriptive Statistics by Group:")
print(arpu_stats)




Group-level summary:
  group_id  total_users  total_revenue  converted_users       ARPU  \
0        A        37188       439673.0             2782  11.822981   
1        B        36262       552916.0             3261  15.247808   

   conversion_rate  
0         0.074809  
1         0.089929  

Two-Proportion Z-test for Conversion Rates:
Z-statistic: -7.456
P-value: 0.00000

Welch's T-test for ARPU (Group A vs. Group B):
t-statistic: -2.989
P-value: 0.00280

ARPU Descriptive Statistics by Group:
  group_id       mean         std  count
0        A  11.822981  120.884832  37188
1        B  15.247808  182.631352  36262


Despite the fact that group B has worse engagement metrics, Group B shows a significantly higher conversion rate than Group A (8.99% vs. 7.48%, Z = -7.456, p < 0.00001) and a significantly higher ARPU (mean $15.25 vs. $11.82, t = -2.989, p = 0.00280). Despite high variability in individual revenue, these results suggest that Group B is more effective at monetizing users.


This could be due to a variety of factors, such as different pricing strategies or more effective in-game purchase prompts.
Further analysis would be needed to determine the specific reasons for these differences and to identify potential opportunities for improving the performance of group A.


Therefore, now will will investigate the pricing strategy and sales.



In [34]:
sales_query = """
SELECT 
  DATE(r.event_timestamp) AS event_date,
  e.group_id,
  e.platform,
  r.package_type,
  SUM(r.dollar_amount) AS total_revenue,
  COUNT(DISTINCT r.user_id) AS distinct_buyers,
  COUNT(*) AS packets_sold
FROM `casedreamgames.case_db.q2_table_ab_test_revenue` r
JOIN `casedreamgames.case_db.q2_table_ab_test_enter` e
  ON r.user_id = e.user_id
GROUP BY event_date, e.group_id, e.platform, r.package_type
ORDER BY event_date, e.group_id, e.platform, r.package_type;
"""
sales = client.query(sales_query).result().to_dataframe()
sales['avg_price'] = sales['total_revenue'] / sales['packets_sold']
sales

/home/yusuf/myenv/lib/python3.12/site-packages/google/cloud/bigquery/table.py:1820: UserWarning:

BigQuery Storage module not found, fetch data with the REST endpoint instead.



,event_date,group_id,platform,package_type,total_revenue,distinct_buyers,packets_sold,avg_price
0,2022-02-08,A,android,coin_pack,4.0,1,1,4.0
1,2022-02-08,A,android,mix_pack,8.0,2,2,4.0
2,2022-02-08,A,ios,coin_pack,16.0,2,4,4.0
3,2022-02-08,A,ios,mix_pack,25.0,4,4,6.25
4,2022-02-08,A,ios,other,122.0,8,14,8.714286
...,...,...,...,...,...,...,...,...
858,2022-04-20,B,android,mix_pack,833.0,22,32,26.03125
859,2022-04-20,B,android,other,428.0,28,35,12.228571
860,2022-04-20,B,ios,coin_pack,1158.0,112,167,6.934132
861,2022-04-20,B,ios,mix_pack,833.0,43,53,15.716981


In [36]:
print(sales.groupby(['group_id', 'platform', 'package_type']).avg_price.describe())

                                count       mean       std       min  \
group_id platform package_type                                         
A        android  coin_pack      72.0   9.158173  2.345601       4.0   
                  mix_pack       72.0  23.926152  7.521337       4.0   
                  other          71.0  13.234298  2.965501  7.575758   
         ios      coin_pack      72.0   9.786923  2.322608       4.0   
                  mix_pack       72.0  20.998649  6.056266      6.25   
                  other          72.0  12.459059  2.144402  8.714286   
B        android  coin_pack      72.0   9.514946  2.695779       4.0   
                  mix_pack       72.0  20.706786  5.160672       9.5   
                  other          72.0  14.132429   2.70284  9.786667   
         ios      coin_pack      72.0   9.194562  2.153003  5.697143   
                  mix_pack       72.0  19.039026  3.759203  8.333333   
                  other          72.0  12.141874  1.926642  8.47

In [37]:
import plotly.express as px

# Plot 1: Average Package Price over time
fig_avg_price = px.line(
    sales,
    x="event_date",
    y="avg_price",
    color="group_id",
    line_dash="platform",
    facet_col="package_type",
    title="Average Package Price by Date, Platform, Test Group, and Package Type",
    labels={
        "event_date": "Event Date",
        "avg_price": "Average Package Price",
        "group_id": "Test Group",
        "platform": "Platform",
        "package_type": "Package Type"
    }
)
fig_avg_price.show()

# Plot 2: Total Revenue over time
fig_total_revenue = px.line(
    sales,
    x="event_date",
    y="total_revenue",
    color="group_id",
    line_dash="platform",
    facet_col="package_type",
    title="Total Revenue of Packets by Date, Platform, Test Group, and Package Type",
    labels={
        "event_date": "Event Date",
        "total_revenue": "Total Revenue",
        "group_id": "Test Group",
        "platform": "Platform",
        "package_type": "Package Type"
    }
)
fig_total_revenue.show()

# Plot 3: Packets Sold over time
fig_packets_sold = px.line(
    sales,
    x="event_date",
    y="packets_sold",
    color="group_id",
    line_dash="platform",
    facet_col="package_type",
    title="Packets Sold by Date, Platform, Test Group, and Package Type",
    labels={
        "event_date": "Event Date",
        "packets_sold": "Packets Sold",
        "group_id": "Test Group",
        "platform": "Platform",
        "package_type": "Package Type"
    }
)
fig_packets_sold.show()

# Plot 4: Distinct Buyers over time
fig_distinct_buyers = px.line(
    sales,
    x="event_date",
    y="distinct_buyers",
    color="group_id",
    line_dash="platform",
    facet_col="package_type",
    title="Distinct Buyers by Date, Platform, Test Group, and Package Type",
    labels={
        "event_date": "Event Date",
        "distinct_buyers": "Distinct Buyers",
        "group_id": "Test Group",
        "platform": "Platform",
        "package_type": "Package Type"
    }
)
fig_distinct_buyers.show()


For Group A, Android users pay an average of $9.16 for coin packs, $23.93 for mix packs, and $13.23 for other packages, while iOS users pay about $9.50, $21.65, and $12.08 respectively. In Group B, coin pack prices are similar (around $9.51 on Android and $8.98 on iOS), mix packs are lower at roughly $20.71 (Android) and $19.04 (iOS), and other packages average $14.13 on Android and $11.81 on iOS. Overall, while coin pack pricing remains consistent, Group A tends to price mix packs slightly higher than Group B, though the differences across groups are relatively modest

In [47]:
import pandas as pd

# Mark users as converted if total_revenue > 0
df_revenue['converted'] = (df_revenue['total_revenue'] > 0).astype(int)

# Aggregate by group_id and package_type
group_pkg = df_revenue.groupby(['group_id', 'package_type']).agg(
    distinct_users=('user_id', 'nunique'),
    converted_users=('converted', 'sum'),
    total_revenue=('total_revenue', 'sum')
).reset_index()

# Calculate overall totals across the entire dataset
overall_users = df_revenue['user_id'].nunique()
overall_converted = df_revenue.loc[df_revenue['converted'] == 1, 'user_id'].nunique()
overall_revenue = df_revenue['total_revenue'].sum()

# Compute percentage contributions
group_pkg['user_perc'] = group_pkg['distinct_users'] / overall_users
group_pkg['converted_user_perc'] = group_pkg['converted_users'] / overall_converted
group_pkg['revenue_perc'] = group_pkg['total_revenue'] / overall_revenue

print("Aggregated Group & Package Data:")
print(group_pkg)


Aggregated Group & Package Data:
  group_id package_type  distinct_users  converted_users  total_revenue  \
0        A    coin_pack            1448             1448       123384.0   
1        A     mix_pack            1889             1889       157463.0   
2        A  no_purchase           34406                0            0.0   
3        A        other            2079             2079       158826.0   
4        B    coin_pack            1865             1865       183328.0   
5        B     mix_pack            2372             2372       186049.0   
6        B  no_purchase           33001                0            0.0   
7        B        other            2275             2275       183539.0   

   user_perc  converted_user_perc  revenue_perc  
0   0.019714             0.239616      0.124305  
1   0.025718             0.312593      0.158639  
2   0.468428             0.000000      0.000000  
3   0.028305             0.344034      0.160012  
4   0.025391             0.308622      0.

In [49]:
import plotly.express as px
import pandas as pd

# Add a "converted" flag: 1 if total_revenue > 0, else 0.
df_revenue['converted'] = (df_revenue['total_revenue'] > 0).astype(int)

# Aggregate revenue data by group_id and package_type.
# Here, total_users is the count of distinct users (nunique on user_id),
# and converted_users is the sum of the "converted" flag.
sales_group = df_revenue.groupby(['group_id', 'package_type']).agg({
    'total_revenue': 'sum',
    'user_id': 'nunique',
    'converted': 'sum'
}).rename(columns={'user_id': 'total_users', 'converted': 'converted_users'})

# Compute percentage contributions relative to overall totals.
sales_group["user_perc"] = sales_group["total_users"] / sales_group["total_users"].sum()
sales_group["revenue_perc"] = sales_group["total_revenue"] / sales_group["total_revenue"].sum()

# Print the aggregated results.
print("Aggregated Sales Group Data:")
print(sales_group)

# Reset index for plotting.
sales_group_reset = sales_group.reset_index()

# Melt the DataFrame to create a long-format version for Plotly.
sales_group_melted = sales_group_reset.melt(
    id_vars=['group_id', 'package_type'], 
    value_vars=['user_perc', 'revenue_perc'], 
    var_name='metric', 
    value_name='percentage'
)

# Create a combined label for x-axis: "Group | Package".
sales_group_melted['group_label'] = sales_group_melted['group_id'].astype(str) + ' | ' + sales_group_melted['package_type']

# Plot the grouped percentages using Plotly.
fig_group = px.bar(
    sales_group_melted, 
    x='group_label', 
    y='percentage', 
    color='metric', 
    barmode='group',
    title="User % and Revenue % by Test Group (Group ID & Package Type)",
    labels={'group_label': 'Test Group (Group ID | Package Type)', 'percentage': 'Percentage'}
)

fig_group.update_layout(xaxis_tickangle=-45)
fig_group.show()


# Print the melted DataFrame.
print("\nMelted Sales Group Data for Plotting:")
print(sales_group_melted)



Aggregated Sales Group Data:
                       total_revenue  total_users  converted_users  user_perc  \
group_id package_type                                                           
A        coin_pack          123384.0         1448             1448   0.018252   
         mix_pack           157463.0         1889             1889   0.023810   
         no_purchase             0.0        34406                0   0.433680   
         other              158826.0         2079             2079   0.026205   
B        coin_pack          183328.0         1865             1865   0.023508   
         mix_pack           186049.0         2372             2372   0.029899   
         no_purchase             0.0        33001                0   0.415970   
         other              183539.0         2275             2275   0.028676   

                       revenue_perc  
group_id package_type                
A        coin_pack         0.124305  
         mix_pack          0.158639  
        


Melted Sales Group Data for Plotting:
   group_id package_type        metric  percentage      group_label
0         A    coin_pack     user_perc    0.018252    A | coin_pack
1         A     mix_pack     user_perc    0.023810     A | mix_pack
2         A  no_purchase     user_perc    0.433680  A | no_purchase
3         A        other     user_perc    0.026205        A | other
4         B    coin_pack     user_perc    0.023508    B | coin_pack
5         B     mix_pack     user_perc    0.029899     B | mix_pack
6         B  no_purchase     user_perc    0.415970  B | no_purchase
7         B        other     user_perc    0.028676        B | other
8         A    coin_pack  revenue_perc    0.124305    A | coin_pack
9         A     mix_pack  revenue_perc    0.158639     A | mix_pack
10        A  no_purchase  revenue_perc    0.000000  A | no_purchase
11        A        other  revenue_perc    0.160012        A | other
12        B    coin_pack  revenue_perc    0.184697    B | coin_pack
13       

The melted data shows that for both groups, a large share of users fall under the "no_purchase" category (43.4% for Group A and 41.6% for Group B), which, as expected, contributes 0% to revenue. In contrast, the paying packages (coin_pack, mix_pack, and other) account for smaller user percentages (roughly 1.8–3.0% per package) but together generate a significant revenue share—Group A's paying packages contribute between 12–16% each, while in Group B they contribute roughly 18–19% each. This indicates that although fewer users make purchases, those purchases drive most of the revenue

In [51]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go

# Aggregate revenue by user (each user appears only once per group)
df_user_revenue = df_revenue.groupby(['user_id', 'group_id'], as_index=False).agg(
    user_total_revenue=('total_revenue', 'sum')
)

# Separate data for Group A and Group B
df_A = df_user_revenue[df_user_revenue['group_id'] == 'A']
df_B = df_user_revenue[df_user_revenue['group_id'] == 'B']

def compute_lorenz_curve(revenue_array):
    # Sort the revenue values in ascending order
    sorted_revenue = np.sort(revenue_array)
    # Compute the cumulative revenue
    cum_revenue = np.cumsum(sorted_revenue)
    total_revenue = cum_revenue[-1]
    # Normalize cumulative revenue to get cumulative percentage of revenue
    lorenz_curve = cum_revenue / total_revenue if total_revenue > 0 else cum_revenue
    # Prepend a 0 to represent the origin (0% users, 0% revenue)
    lorenz_curve = np.insert(lorenz_curve, 0, 0)
    # Generate the cumulative percentage of users (x-axis)
    n = len(sorted_revenue)
    cum_users = np.linspace(0, 1, n + 1)
    return cum_users, lorenz_curve

# Compute Lorenz curve coordinates for each group
x_A, y_A = compute_lorenz_curve(df_A['user_total_revenue'].values)
x_B, y_B = compute_lorenz_curve(df_B['user_total_revenue'].values)

# Create the plot using Plotly
fig = go.Figure()
fig.add_trace(go.Scatter(x=x_A, y=y_A, mode='lines+markers', name='Group A'))
fig.add_trace(go.Scatter(x=x_B, y=y_B, mode='lines+markers', name='Group B'))
# Add the 45° equality line for reference
fig.add_trace(go.Scatter(x=[0, 1], y=[0, 1], mode='lines', 
                         line=dict(dash='dash', color='black'),
                         name='Equality Line'))

fig.update_layout(
    title="Lorenz Curve for Revenue by Group",
    xaxis_title="Cumulative Percentage of Users",
    yaxis_title="Cumulative Percentage of Revenue",
    xaxis=dict(tickformat=".0%"),
    yaxis=dict(tickformat=".0%"),
    height=700
)
fig.show()


The curvature is less for group B, this means the users in group b  tend to have a little bit more equally contribute to the total revenue
We will investigate the effect of pricing on demand, we are defining the demand as the ratio of purchases oon total users

In [56]:

query_avg_price_and_demand = """
WITH 
  -- Get active sessions from the session table.
  active_sessions AS (
    SELECT 
      DATE(event_timestamp) AS event_date,
      user_id,
      platform
    FROM `casedreamgames.case_db.q2_table_ab_test_session`
    GROUP BY event_date, user_id, platform
  ),
  -- Join active sessions with test entries to assign group_id.
  active_users AS (
    SELECT 
      a.event_date,
      te.user_id,
      te.group_id,
      a.platform
    FROM active_sessions a
    JOIN `casedreamgames.case_db.q2_table_ab_test_enter` te
      ON a.user_id = te.user_id
  ),
  daily_active AS (
    SELECT 
      event_date,
      group_id,
      platform,
      COUNT(DISTINCT user_id) AS total_active_users
    FROM active_users
    GROUP BY event_date, group_id, platform
  ),
  -- Get revenue data.
  revenue_data AS (
    SELECT 
      DATE(event_timestamp) AS event_date,
      user_id,
      platform,
      package_type,
      dollar_amount
    FROM `casedreamgames.case_db.q2_table_ab_test_revenue`
  ),
  -- Also get test entry info to assign group_id to revenue events.
  test_info AS (
    SELECT 
      user_id,
      group_id,
      platform
    FROM `casedreamgames.case_db.q2_table_ab_test_enter`
  )
  
SELECT 
  r.event_date,
  r.platform,
  t.group_id,
  r.package_type,
  AVG(r.dollar_amount) AS avg_price,
  COUNT(DISTINCT r.user_id) AS buyers,
  d.total_active_users,
  COUNT(DISTINCT r.user_id) * 1.0 / d.total_active_users AS demand
FROM revenue_data r
JOIN test_info t
  ON r.user_id = t.user_id
JOIN daily_active d
  ON r.event_date = d.event_date 
  AND r.platform = d.platform 
  AND t.group_id = d.group_id
GROUP BY r.event_date, r.platform, t.group_id, r.package_type, d.total_active_users
ORDER BY r.event_date, r.platform, t.group_id, r.package_type;
"""
df_demand = client.query(query_avg_price_and_demand).result().to_dataframe()
df_demand




/home/yusuf/myenv/lib/python3.12/site-packages/google/cloud/bigquery/table.py:1820: UserWarning:

BigQuery Storage module not found, fetch data with the REST endpoint instead.



,event_date,platform,group_id,package_type,avg_price,buyers,total_active_users,demand
0,2022-02-08,android,A,coin_pack,4.000000,1,298,0.003356
1,2022-02-08,android,A,mix_pack,4.000000,2,298,0.006711
2,2022-02-08,android,B,coin_pack,4.000000,1,351,0.002849
3,2022-02-08,android,B,mix_pack,9.833333,4,351,0.011396
4,2022-02-08,android,B,other,11.000000,2,351,0.005698
...,...,...,...,...,...,...,...,...
858,2022-04-20,ios,A,mix_pack,32.533333,28,7576,0.003696
859,2022-04-20,ios,A,other,10.048780,38,7576,0.005016
860,2022-04-20,ios,B,coin_pack,6.934132,112,6775,0.016531
861,2022-04-20,ios,B,mix_pack,15.716981,43,6775,0.006347


In [57]:
# Plotting the average price and demand over time
import plotly.express as px

fig_avg_price = px.line(
    df_demand,
    x="event_date",
    y="avg_price",
    color="group_id",
    line_dash="platform",
    facet_col="package_type",
    title="Average Package Price by Date, Platform, Test Group, and Package Type",
    labels={
        "event_date": "Event Date",
        "avg_price": "Average Package Price",
        "group_id": "Test Group",
        "platform": "Platform",
        "package_type": "Package Type"
    }
)
fig_avg_price.show()

fig_demand_line = px.line(
    df_demand,
    x="event_date",
    y="demand",
    color="group_id",
    line_dash="platform",
    facet_col="package_type",
    title="Demand (Ratio of Buyers to The active users) by Date, Platform, Test Group, and Package Type",
    labels={
        "event_date": "Event Date",
        "demand": "Ratio of Buyers (Demand)",
        "group_id": "Test Group",
        "platform": "Platform",
        "package_type": "Package Type"
    }
)
fig_demand_line.show()


Now, we perform a regression analysis for each combination of group_id, platform, and package_type.
The target is the percentage change in demand, and the exogenous variable is the percentage change in average price.

In [63]:
import pandas as pd
import statsmodels.api as sm

df_demand['event_date'] = pd.to_datetime(df_demand['event_date'])
df_demand = df_demand.sort_values('event_date')

# Containers to store results.
results_beta = []  
results_tscore = []  

unique_combinations = df_demand[['group_id', 'platform', 'package_type']].drop_duplicates()

for _, combo in unique_combinations.iterrows():
    grp = combo['group_id']
    plat = combo['platform']
    pack = combo['package_type']
    
    # Subset the data for the current combination
    df_subset = df_demand[
        (df_demand['group_id'] == grp) &
        (df_demand['platform'] == plat) &
        (df_demand['package_type'] == pack)
    ].copy()
    
    # Sort by event_date and skip initial rows if needed (here skipping the first 5 observations)
    df_subset = df_subset.sort_values('event_date').iloc[5:]
    
    # Calculate percentage changes
    df_subset['price_pct_change'] = df_subset['avg_price'].pct_change()
    df_subset['demand_pct_change'] = df_subset['demand'].pct_change()
    
    # Drop rows with NA (from pct_change)
    df_subset = df_subset.dropna()
    
    # Check if we have enough data points for regression
    if df_subset.shape[0] < 10:
        print(f"Skipping regression for Group {grp}, Platform {plat}, Package {pack} due to insufficient data (n={df_subset.shape[0]}).")
        continue
    
    # Define independent (X) and dependent (y) variables
    X = df_subset['price_pct_change']
    y = df_subset['demand_pct_change']
    
    # Add constant for intercept
    X = sm.add_constant(X)
    
    # Fit the regression model
    model = sm.OLS(y, X).fit()
    
    # Extract the coefficient, t-score, and p-value for price_pct_change
    beta = model.params['price_pct_change']
    t_score = model.tvalues['price_pct_change']
    p_value = model.pvalues['price_pct_change']
    n_obs = df_subset.shape[0]
    
    result = {
        'group_id': grp,
        'platform': plat,
        'package_type': pack,
        'beta': beta,
        't_score': t_score,
        'p_value': p_value,
        'n_obs': n_obs
    }
    
    results_beta.append(result)
    results_tscore.append(result)
    
    print(f"Regression Summary for Group {grp}, Platform {plat}, Package {pack}:")
    print(model.summary())
    print("\n" + "="*80 + "\n")

# Sort results by absolute beta coefficient (largest first)
sorted_by_beta = sorted(results_beta, key=lambda x: abs(x['beta']), reverse=True)
# Sort results by absolute t_score (largest first)
sorted_by_tscore = sorted(results_tscore, key=lambda x: abs(x['t_score']), reverse=True)



Regression Summary for Group A, Platform android, Package coin_pack:
                            OLS Regression Results                            
Dep. Variable:      demand_pct_change   R-squared:                       0.010
Model:                            OLS   Adj. R-squared:                 -0.005
Method:                 Least Squares   F-statistic:                    0.6679
Date:                Sat, 22 Feb 2025   Prob (F-statistic):              0.417
Time:                        18:54:44   Log-Likelihood:                 3.6317
No. Observations:                  66   AIC:                            -3.263
Df Residuals:                      64   BIC:                             1.116
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                       coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------

In [64]:
print("Top Results by Absolute Beta:")
for res in sorted_by_beta[:3]:
    print(res)

print("\nTop Results by Absolute t-score:")
for res in sorted_by_tscore[:3]:
    print(res)


Top Results by Absolute Beta:
{'group_id': 'A', 'platform': 'ios', 'package_type': 'other', 'beta': np.float64(0.12810442397688362), 't_score': np.float64(0.6457438917670089), 'p_value': np.float64(0.5207538448807456), 'n_obs': 66}
{'group_id': 'A', 'platform': 'android', 'package_type': 'other', 'beta': np.float64(0.10205770200036454), 't_score': np.float64(0.4259368951419462), 'p_value': np.float64(0.6716055304199571), 'n_obs': 65}
{'group_id': 'B', 'platform': 'android', 'package_type': 'other', 'beta': np.float64(-0.08552039158754886), 't_score': np.float64(-0.4335719754153053), 'p_value': np.float64(0.6660572458362015), 'n_obs': 66}

Top Results by Absolute t-score:
{'group_id': 'B', 'platform': 'ios', 'package_type': 'coin_pack', 'beta': np.float64(-0.05007874844793473), 't_score': np.float64(-0.8966304663021225), 'p_value': np.float64(0.37327651375033266), 'n_obs': 66}
{'group_id': 'A', 'platform': 'android', 'package_type': 'coin_pack', 'beta': np.float64(-0.06970756593685339),

The regression results of price change on demand change show that the coefficient of the price change variable is not statistically significant for any of the groups, platforms, or package types. This implies that the price changes do not have a significant impact on the demand changes in the observed data.
However, most of them are negative, indicating that an increase in price is associated with a decrease in demand

In [76]:
query = """
WITH 
  revenue_data AS (
    SELECT
      DATE(r.event_timestamp) AS purchase_date,
      e.group_id,
      COUNT(*) AS total_purchases,
      AVG(r.dollar_amount) AS avg_purchase_amount
    FROM `casedreamgames.case_db.q2_table_ab_test_revenue` r
    JOIN `casedreamgames.case_db.q2_table_ab_test_enter` e
      ON r.user_id = e.user_id
    GROUP BY purchase_date, e.group_id
    ORDER BY purchase_date, e.group_id
  ),
  session_counts AS (
    SELECT 
      DATE(s.event_timestamp) AS session_date,
      t.group_id,
      COUNT(*) AS total_sessions
    FROM `casedreamgames.case_db.q2_table_ab_test_session` s
    JOIN `casedreamgames.case_db.q2_table_ab_test_enter` t
      ON s.user_id = t.user_id
    GROUP BY session_date, t.group_id
  )
SELECT 
  r.purchase_date,
  r.group_id,
  r.total_purchases,
  r.avg_purchase_amount,
  s.total_sessions
FROM revenue_data r
LEFT JOIN session_counts s
  ON r.purchase_date = s.session_date 
  AND r.group_id = s.group_id
ORDER BY r.purchase_date, r.group_id;

"""
df_purchase = client.query(query).result().to_dataframe()
df_purchase['purchase_rate'] = df_purchase['total_purchases'] / df_purchase['total_sessions']
print(df_purchase.groupby('group_id')[['purchase_rate','avg_purchase_amount']].describe())

/home/yusuf/myenv/lib/python3.12/site-packages/google/cloud/bigquery/table.py:1820: UserWarning:

BigQuery Storage module not found, fetch data with the REST endpoint instead.



         purchase_rate                                                    \
                 count      mean       std       min       25%       50%   
group_id                                                                   
A                 72.0  0.000245  0.000054  0.000084   0.00021   0.00024   
B                 72.0  0.000448  0.000108  0.000122  0.000387  0.000432   

                             avg_purchase_amount                       \
               75%       max               count       mean       std   
group_id                                                                
A         0.000284  0.000388                72.0  13.444580  1.937095   
B         0.000485   0.00078                72.0  12.639493  1.405702   

                                                                
               min        25%        50%        75%        max  
group_id                                                        
A         7.000000  12.340026  13.419553  14.670027  19.73

In [79]:
import plotly.express as px

# Plot 1: Total Purchases by Date and Group
fig_total_purchases = px.line(
    df_purchase,
    x="purchase_date",
    y="total_purchases",
    color="group_id",
    title="Total Purchases by Date and Group",
    labels={"purchase_date": "Purchase Date", "total_purchases": "Total Purchases", "group_id": "Test Group"}
)
fig_total_purchases.show()

# Plot 2: Average Purchase Amount by Date and Group
fig_avg_purchase = px.line(
    df_purchase,
    x="purchase_date",
    y="avg_purchase_amount",
    color="group_id",
    title="Average Purchase Amount by Date and Group",
    labels={"purchase_date": "Purchase Date", "avg_purchase_amount": "Avg Purchase Amount ($)", "group_id": "Test Group"}
)
fig_avg_purchase.show()

# Plot 3: Total Sessions by Date and Group
fig_total_sessions = px.line(
    df_purchase,
    x="purchase_date",
    y="total_sessions",
    color="group_id",
    title="Total Sessions by Date and Group",
    labels={"purchase_date": "Purchase Date", "total_sessions": "Total Sessions", "group_id": "Test Group"}
)
fig_total_sessions.show()

# Plot 4: Purchase Rate by Date and Group
fig_purchase_rate = px.line(
    df_purchase,
    x="purchase_date",
    y="purchase_rate",
    color="group_id",
    title="Purchase Rate by Date and Group",
    labels={"purchase_date": "Purchase Date", "purchase_rate": "Purchase Rate", "group_id": "Test Group"}
)
fig_purchase_rate.show()


The results show that Group B exhibits a higher purchase rate per session (mean of 0.000448) compared to Group A (mean of 0.000245), indicating that, on average, users in Group B make a purchase more frequently relative to the number of sessions played. However, the average purchase amount is slightly lower in Group B (mean $12.64) compared to Group A (mean $13.44), suggesting that while Group B converts more often, the value per purchase is marginally higher in Group A. This is the reason why Group B has better monetization metrics.  

This could be because Group B’s variant may better prompt or incentivize players to buy items when they face in-game challenges—through enhanced in-game prompts, targeted offers, or personalized rewards—even though the pricing is similar; these mechanisms help drive up the conversion rate and overall revenue despite lower engagement metrics